# LIVRABLE 1 - Classification binaire (Projet LEYENDA)

# Objectif : Distinguer les photos naturelles (couleur ou noir et blanc) des autres types d'images (peintures, textes, dessins, schémas...)

CHANGER EN PNG, RESIZE
MODELE RGB
MODELE NB
RECONNAITRE PHOTO PEINTURE


In [ ]:
# =======================
# 🚩 Partie 1 : Initialisation & Vérification GPU
# =======================

# Importations générales
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

# ✅ Vérification de la disponibilité GPU (TensorFlow GPU Metal)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("[INFO] GPU détecté :", gpus)
else:
    print("[WARNING] Pas de GPU détecté. Vérifie tensorflow-metal.")

In [ ]:
# =======================
# 📂 Partie 2 : Organisation initiale des dossiers
# =======================

# Chemins des dossiers
source_dir = "/Users/clementfornes/Downloads/leyenda project/images"
working_dir = "/Users/clementfornes/Downloads/leyenda project/images_work"

# Auto-détection des catégories présentes dans source_dir
categories = [d for d in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, d))]

# Création du dossier de travail et copie des images originales
os.makedirs(working_dir, exist_ok=True)

for category in categories:
    src_path = os.path.join(source_dir, category)
    dst_path = os.path.join(working_dir, category)

    if not os.path.exists(dst_path):
        shutil.copytree(src_path, dst_path)
        print(f"[COPIED] Catégorie : {category}")
    else:
        print(f"[ALREADY EXISTS] Catégorie : {category}")

In [ ]:
# =======================
# 🎨 Partie 3 : Tri automatique en Couleur et Noir & Blanc
# =======================

# Dossier de travail et catégories
categories = ["Painting", "Photo", "Schematics", "Sketch", "Text"]

# Fonction pour détecter si une image est en niveau de gris
def is_grayscale(img_path):
    img = Image.open(img_path).convert("RGB")
    np_img = np.array(img)
    return np.all(np_img[:,:,0] == np_img[:,:,1]) and np.all(np_img[:,:,1] == np_img[:,:,2])

# Tri des images selon la couleur
for category in categories:
    path = os.path.join(working_dir, category)

    # Création des sous-dossiers rgb et nb
    rgb_path = os.path.join(path, "rgb")
    nb_path = os.path.join(path, "nb")
    os.makedirs(rgb_path, exist_ok=True)
    os.makedirs(nb_path, exist_ok=True)

    files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]

    for file_name in tqdm(files, desc=f"[CLASSIFY] {category}", unit="image"):
        file_path = os.path.join(path, file_name)
        base_name, ext = os.path.splitext(file_name)

        try:
            img = Image.open(file_path).convert("RGB")
            np_img = np.array(img)

            # Vérifier si l'image est en noir et blanc ou couleur
            dest_folder = nb_path if is_grayscale(file_path) else rgb_path

            # Sauvegarde systématique en PNG
            new_filename = base_name + ".png"
            dest_path = os.path.join(dest_folder, new_filename)
            img.save(dest_path, "PNG")

            # Suppression des fichiers originaux non nécessaires
            if os.path.abspath(dest_path) != os.path.abspath(file_path):
                os.remove(file_path)

        except Exception as e:
            print(f"[ERROR] Fichier {file_path} : {e}")

In [ ]:
# =======================
# 🎨 Partie 4 : Normalisation de la taille des images 
# =======================
for category in categories:
    path = os.path.join(working_dir, category)
    resized_path = os.path.join(path, "resized", "rgb")
    os.makedirs(resized_path, exist_ok=True)

    files = [f for f in os.listdir(os.path.join(path, "rgb")) if os.path.isfile(os.path.join(path, "rgb", f))]
    for file_name in tqdm(files, desc=f"[RESIZE] {category}", unit="image"):
        file_path = os.path.join(path, "rgb", file_name)
        base_name, ext = os.path.splitext(file_name)

        try:
            img = Image.open(file_path).convert("RGB")
            img_resized = img.resize((256, 256), Image.Resampling.LANCZOS)
            new_filename = base_name + ".png"
            dest_path = os.path.join(resized_path, new_filename)
            img_resized.save(dest_path, "PNG")

            # Suppression des fichiers originaux non nécessaires
            if os.path.abspath(dest_path) != os.path.abspath(file_path):
                os.remove(file_path)

        except Exception as e:
            print(f"[ERROR] Fichier {file_path} : {e}")
# Normalisation de la taille des images en noir et blanc
for category in categories:
    path = os.path.join(working_dir, category)
    resized_path = os.path.join(path, "resized", "nb")
    os.makedirs(resized_path, exist_ok=True)

    files = [f for f in os.listdir(os.path.join(path, "nb")) if os.path.isfile(os.path.join(path, "nb", f))]
    for file_name in tqdm(files, desc=f"[RESIZE] {category}", unit="image"):
        file_path = os.path.join(path, "nb", file_name)
        base_name, ext = os.path.splitext(file_name)

        try:
            img = Image.open(file_path).convert("RGB")
            img_resized = img.resize((256, 256), Image.Resampling.LANCZOS)
            new_filename = base_name + ".png"
            dest_path = os.path.join(resized_path, new_filename)
            img_resized.save(dest_path, "PNG")

            # Suppression des fichiers originaux non nécessaires
            if os.path.abspath(dest_path) != os.path.abspath(file_path):
                os.remove(file_path)

        except Exception as e:
            print(f"[ERROR] Fichier {file_path} : {e}")
# =======================
print("[INFO] Tri et redimensionnement des images terminés.")
# Display des images triées (nombre limité à 12)
def display_images_from_folder(folder_path):
    images = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.png')]
    images = images[:12]  # Limiter à 12 images
    plt.figure(figsize=(15, 10))
    for i, img_path in enumerate(images):
        img = Image.open(img_path)
        plt.subplot(3, 4, i + 1)
        plt.imshow(img)
        plt.axis('off')
    plt.show()
# Affichage des images triées
for category in categories:
    path = os.path.join(working_dir, category, "resized", "rgb")
    print(f"[DISPLAY] Images de la catégorie : {category}")
    display_images_from_folder(path)
# =======================
               


In [ ]:
import os
import shutil
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# Dossiers d'entraînement et validation
train_dir_rgb = os.path.join(working_dir, "train", "rgb")
val_dir_rgb = os.path.join(working_dir, "val", "rgb")
train_dir_nb = os.path.join(working_dir, "train", "nb")
val_dir_nb = os.path.join(working_dir, "val", "nb")
# Création des dossiers d'entraînement et validation
os.makedirs(train_dir_rgb, exist_ok=True)
os.makedirs(val_dir_rgb, exist_ok=True)
os.makedirs(train_dir_nb, exist_ok=True)
os.makedirs(val_dir_nb, exist_ok=True)
# Création des sous-dossiers pour chaque catégorie
for category in categories:
    os.makedirs(os.path.join(train_dir_rgb, category), exist_ok=True)
    os.makedirs(os.path.join(val_dir_rgb, category), exist_ok=True)
    os.makedirs(os.path.join(train_dir_nb, category), exist_ok=True)
    os.makedirs(os.path.join(val_dir_nb, category), exist_ok=True)
# Copie des images dans les dossiers d'entraînement et validation
# Copie des images RGB et NB (noir et blanc) dans les dossiers d'entraînement et validation

for category in categories:
    # RGB
    rgb_path = os.path.join(working_dir, category, "resized", "rgb")
    files_rgb = [f for f in os.listdir(rgb_path) if f.endswith('.png')]
    train_rgb, val_rgb = train_test_split(files_rgb, test_size=0.2, random_state=42)

    for file_name in tqdm(train_rgb, desc=f"[COPY TRAIN] {category} RGB", unit="image"):
        src_path = os.path.join(rgb_path, file_name)
        dst_path = os.path.join(train_dir_rgb, category, file_name)
        shutil.copy(src_path, dst_path)

    for file_name in tqdm(val_rgb, desc=f"[COPY VAL] {category} RGB", unit="image"):
        src_path = os.path.join(rgb_path, file_name)
        dst_path = os.path.join(val_dir_rgb, category, file_name)
        shutil.copy(src_path, dst_path)

    # NB
    nb_path = os.path.join(working_dir, category, "resized", "nb")
    files_nb = [f for f in os.listdir(nb_path) if f.endswith('.png')]
    train_nb, val_nb = train_test_split(files_nb, test_size=0.2, random_state=42)

    for file_name in tqdm(train_nb, desc=f"[COPY TRAIN] {category} NB", unit="image"):
        src_path = os.path.join(nb_path, file_name)
        dst_path = os.path.join(train_dir_nb, category, file_name)
        shutil.copy(src_path, dst_path)

    for file_name in tqdm(val_nb, desc=f"[COPY VAL] {category} NB", unit="image"):
        src_path = os.path.join(nb_path, file_name)
        dst_path = os.path.join(val_dir_nb, category, file_name)
        shutil.copy(src_path, dst_path)
# =======================
print("[INFO] Copie des images terminée.")
# =======================
# Affichage des images triées
for category in categories:
    path = os.path.join(working_dir, category, "resized", "rgb")
    print(f"[DISPLAY] Images de la catégorie : {category}")
    display_images_from_folder(path)
# =======================
# Affichage des images triées
for category in categories:
    path = os.path.join(working_dir, category, "resized", "nb")
    print(f"[DISPLAY] Images de la catégorie : {category}")
    display_images_from_folder(path)

In [ ]:
# =======================
# 📦 Partie 5 : Chargement et Prétraitement des données
# =======================

# Chemins des dossiers d'entraînement et de validation
train_dir_rgb = os.path.join(working_dir, "train", "rgb")
train_dir_nb = os.path.join(working_dir, "train", "nb")
val_dir_rgb = os.path.join(working_dir, "val", "rgb")
val_dir_nb = os.path.join(working_dir, "val", "nb")
# Taille d'image cible
img_height = 256    
img_width = 256
# Nombre de classes 
num_classes = len(categories) * 2  # RGB + NB
# Chemin du dossier de sauvegarde des modèles
model_dir = os.path.join(working_dir, "models")
os.makedirs(model_dir, exist_ok=True)
# Chemin du dossier de sauvegarde des résultats
results_dir = os.path.join(working_dir, "results")
os.makedirs(results_dir, exist_ok=True)
# Chemin du fichier de log
log_file = os.path.join(results_dir, "training_log.txt")
# Ouverture du fichier de log
log = open(log_file, "w")
# Fonction pour charger et prétraiter les images
def load_and_preprocess_image(file_path):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, [img_height, img_width])
    img = tf.cast(img, tf.float32) / 255.0
    return img
# Fonction pour charger les étiquettes
def load_and_preprocess_label(label):
    label = tf.strings.to_number(label, tf.int32)
    return label
# Fonction pour créer un dataset à partir d'un dossier
def create_dataset_from_directory(directory, batch_size=32):
    file_paths = []
    labels = []
    for label, category in enumerate(categories):
        category_dir_rgb = os.path.join(directory, category)
        category_dir_nb = os.path.join(directory, category)
        for img_file in os.listdir(category_dir_rgb):
            file_paths.append(os.path.join(category_dir_rgb, img_file))
            labels.append(label)
        for img_file in os.listdir(category_dir_nb):
            file_paths.append(os.path.join(category_dir_nb, img_file))
            labels.append(label + len(categories))  # Décalage pour les images NB
    file_paths = tf.convert_to_tensor(file_paths)
    labels = tf.convert_to_tensor(labels)
    dataset = tf.data.Dataset.from_tensor_slices((file_paths, labels))
    dataset = dataset.map(lambda x, y: (load_and_preprocess_image(x), load_and_preprocess_label(y)))
    dataset = dataset.shuffle(buffer_size=len(file_paths))
    dataset = dataset.batch(batch_size)
    return dataset
# Création des datasets d'entraînement et de validation
batch_size = 32
train_dataset = create_dataset_from_directory(train_dir_rgb, batch_size)
train_dataset_nb = create_dataset_from_directory(train_dir_nb, batch_size)
val_dataset = create_dataset_from_directory(val_dir_rgb, batch_size)
val_dataset_nb = create_dataset_from_directory(val_dir_nb, batch_size)
# Fusion des datasets RGB et NB
train_dataset = train_dataset.concatenate(train_dataset_nb)
val_dataset = val_dataset.concatenate(val_dataset_nb)
# Affichage du nombre d'images dans chaque dataset
print(f"[INFO] Nombre d'images d'entraînement : {len(train_dataset)}")
print(f"[INFO] Nombre d'images de validation : {len(val_dataset)}")
# =======================
# 📊 Partie 6 : Visualisation des données
# =======================
# Fonction pour afficher les images et leurs étiquettes
def display_images(dataset, num_images=12):
    plt.figure(figsize=(15, 10))
    for i, (img, label) in enumerate(dataset.take(num_images)):
        plt.subplot(3, 4, i + 1)
        plt.imshow(img[0])
        plt.title(f"Label: {label[0].numpy()}")
        plt.axis('off')
    plt.show()
# Affichage des images d'entraînement
print("[DISPLAY] Images d'entraînement")
display_images(train_dataset)
# Affichage des images de validation
print("[DISPLAY] Images de validation")
display_images(val_dataset)
# =======================